In [1]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
path_file_input = "G:/data/NLP/Otvety.txt"
path_file_output = "G:/data/NLP/prepared_answers.txt"

In [3]:
assert True

#Small preprocess of the answers

question = None
written = False

with open(path_file_output, "w", encoding="utf8") as fout:
    with open(path_file_input, "r", encoding="utf8") as fin:
        for line in tqdm(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

In [4]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [5]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open(path_file_input, "r", encoding="utf8") as fin:
    for line in tqdm(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

In [6]:
sentences = [i for i in sentences if len(i) > 2]
modelFT = FastText(sentences=sentences, size=100, min_count=1, window=5)
modelFT.save("ft_model")

In [8]:
modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open(path_file_output, "r", encoding="utf8") as f:
    for line in tqdm(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

ft_index.build(10)
ft_index.save('speaker.ann')

D:\anaconda3\envs\NLP\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  from ipykernel import kernelapp as app
D:\anaconda3\envs\NLP\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


True

In [9]:
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann') 

True

In [10]:
ft_index.get_nns_by_vector(np.zeros(100), 2)

[20216, 26555]

In [11]:
shop_data = pd.read_csv("ProductsDataset.csv")


shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указанный, ра..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новое, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [13]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in idxs]
positive_texts = [" ".join(val) for val in shop_data['text'].values]

In [14]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, stratify=labels,
                                                    random_state=13)

In [17]:
x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

In [18]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9798809861150467

In [19]:
assert True
import pickle

with open("index_speaker.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

In [21]:
np.mean(tfidf_vect.idf_)

10.921957563612375

In [22]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}
idfs

{'юбка': 9.376426898696234,
 'белый': 6.978531625897864,
 '46': 11.248229075597825,
 'одеть': 11.248229075597825,
 'размер': 11.248229075597825,
 'примерка': 11.248229075597825,
 'подкинуть': 11.248229075597825,
 'артем': 11.248229075597825,
 'подьезда': 11.248229075597825,
 'смотреть': 11.248229075597825,
 'база': 9.995466107102457,
 'вещий': 11.248229075597825,
 'платье': 11.248229075597825,
 'заказывать': 11.248229075597825,
 'носиться': 10.842763967489661,
 'резинка': 11.248229075597825,
 'практически': 11.248229075597825,
 'люба': 11.248229075597825,
 'бедро': 11.248229075597825,
 'забирать': 11.248229075597825,
 'молоков': 11.248229075597825,
 'лот': 11.248229075597825,
 'интересный': 11.248229075597825,
 'материал': 11.248229075597825,
 'потип': 11.248229075597825,
 'клеёнчатый': 11.248229075597825,
 'httpwwwworldartru': 11.248229075597825,
 'вроде': 11.248229075597825,
 'пиджак': 11.248229075597825,
 'мужской': 11.248229075597825,
 'новый': 11.248229075597825,
 'одетый': 11.248

In [23]:
ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

midf = np.mean(tfidf_vect.idf_)

index_map_shop = {}
counter = 0

for i in tqdm(range(len(shop_data))):
    n_ft = 0
    index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
    vector_ft = np.zeros(100)
    for word in shop_data.loc[i, "text"]:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

ft_index_shop.build(10)
ft_index_shop.save('shop.ann')

D:\anaconda3\envs\NLP\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  del sys.path[0]
D:\anaconda3\envs\NLP\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


True

In [24]:
assert True
import pickle

with open("index_shop.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [25]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [26]:
ft_index_shop.get_nns_by_vector(np.zeros(100), 1, include_distances=True)

([102], [1.4142135381698608])

In [31]:
updater = Updater(token='1145420328:AAEQhT_v04e5cJIqQ-JOfccgVzDqtCiqLRg') # Токен API к Telegram
dispatcher = updater.dispatcher


def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Добрый день')

def textMessage(bot, update):
    
    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)
    
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    if distances[0] > 0.2:
        print(distances[0])
        bot.send_message(chat_id=update.message.chat_id, text="Моя твоя не понимать")
        return
    bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])
        
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

D:\anaconda3\envs\NLP\lib\site-packages\ipykernel_launcher.py:1: TelegramDeprecationWarning: Old Handler API is deprecated - see https://git.io/fxJuV for details
  """Entry point for launching an IPython kernel.


InvalidToken: Invalid token